In [ ]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
genai.configure()

In [ ]:
system_message = "You are a helpful and friendly assistant that\
    helps the user with their queries.\
        You essentially serve the same purpose as Alexa or Siri.\
            Avoid going too in-depth for techinical or complicated questions such as coding, maths or science."

In [ ]:
def chat_with_openai(message, history):
    messages =[{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages, stream = True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response  

In [ ]:
def chat_with_claude(message, history):
    
    def _as_text(x):
        if isinstance(x, str):
            return x
        if isinstance(x, list):
            parts = []
            for p in x:
                if isinstance(p, str):
                    parts.append(p)
                elif isinstance(p, dict):
                    # common shapes: {"type":"text","text":"..."} or {"text":"..."}
                    parts.append(p.get("text", ""))
            return "".join(parts)
        return str(x) if x is not None else ""

    sanitized = []

    if history and isinstance(history[0], dict):
        # type="messages" style history
        for m in history:
            role = m.get("role")
            if role in ("user", "assistant"):
                sanitized.append({"role": role, "content": _as_text(m.get("content", ""))})
    else:
        # default ChatInterface history: list of (user, assistant) tuples
        for u, a in (history or []):
            if u:
                sanitized.append({"role": "user", "content": _as_text(u)})
            if a:
                sanitized.append({"role": "assistant", "content": _as_text(a)})

    # last turn from the UI
    sanitized.append({"role": "user", "content": _as_text(message)})

    # optional: drop empty messages (Claude dislikes empty content)
    sanitized = [m for m in sanitized if m["content"].strip()]

    stream = claude.messages.create(
        model= "claude-3-5-haiku-latest",
        max_tokens=4000,
        system=system_message,              # must be a plain string
        messages=sanitized,
        stream=True,
    )

    response = ""
    for chunk in stream:
        # handle only text deltas
        if getattr(chunk, "type", "") == "content_block_delta":
            text = getattr(getattr(chunk, "delta", None), "text", None)
            if text:
                response += text
                yield response


In [ ]:

def chat_with_gemini(message, history):
    
    model = genai.GenerativeModel(
        'gemini-2.5-flash-lite',  
        system_instruction=system_message 
    )
    
    gemini_history = []
    for msg in history:
        if msg["role"] == "user":
            gemini_history.append({"role": "user", "parts": [msg["content"]]})
        elif msg["role"] == "assistant":
            gemini_history.append({"role": "model", "parts": [msg["content"]]})
    
    chat = model.start_chat(history=gemini_history)
    response_stream = chat.send_message(message, stream=True)
    
    response = ""
    for chunk in response_stream:
        if chunk.text:
            response += chunk.text
            yield response

In [ ]:
def multi_llm_chat(message, history, model):
    if model == "OpenAI":
        yield from chat_with_openai(message, history)
    elif model == "Claude":
        yield from chat_with_claude(message, history)
    elif model == "Gemini":
        yield from chat_with_gemini(message, history)


In [ ]:
def create_custom_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Multi-LLM Assistant")
        gr.Markdown("Select your preferred AI model below")
        
        model_dropdown = gr.Dropdown(
            choices=["OpenAI", "Claude", "Gemini"], 
            value=None,
            label="Models",
            info="You must select a model"
        )
        
        gr.Markdown("Start your converstion once you've selected a model")
        
        # Create the chat interface with the model as additional input
        chat = gr.ChatInterface(
            fn=multi_llm_chat,
            additional_inputs=[model_dropdown],
            type="messages"
        )
    
    return demo

In [ ]:
view = create_custom_interface()
view.launch(inbrowser=True)

In [ ]:
# view = gr.ChatInterface(
#     fn=multi_llm_chat,
#     additional_inputs=[
#         gr.Dropdown(choices=["OpenAI", "Claude", "Gemini"], value="None", label="Select model")
#     ],
#     type="messages"
# )

# view.launch(inbrowser=True)